<a href="https://colab.research.google.com/github/dkgin/fate3/blob/main/weather_line.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#連動自己帳號的 Google Drive，並在指定目錄下建立 ngrok 相關目錄
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!mkdir -p /drive
#umount /drive
!mount --bind /content/drive/My\ Drive /drive
!mkdir -p /drive/ngrok-ssh
!mkdir -p ~/.ssh

Mounted at /content/drive


In [2]:
#下載 ngrok 的 ssh 檔案，並自動解壓縮放到指定的目錄下，目的在於要讓 ngrok 網站認證身份。
!mkdir -p /drive/ngrok-ssh
%cd /drive/ngrok-ssh
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip
!cp /drive/ngrok-ssh/ngrok /ngrok
!chmod +x /ngrok

/drive/ngrok-ssh
--2024-01-25 14:52:59--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  20.9MB/s    in 0.6s    

2024-01-25 14:53:00 (20.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip


In [3]:
#輸入 ngrok token
!/ngrok authtoken 2aU1e79cEJaC0o3AkbfbR1c1nDi_5EMpYx9u2jPtQq6ZC4Zud

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [4]:
#安裝 flask_ngrok 函式庫
!pip install flask_ngrok
#安裝 LINE BOT 函式庫
!pip install line-bot-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 748.7/748.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.14
    Uninstalling pydantic-1.10.14:
      Successfully uninstalled pydantic-1.10.14
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-m

In [5]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request
# 載入 LINE Message API 相關函式庫
from linebot import LineBotApi, WebhookHandler
from linebot.models import MessageEvent, TextMessage, TextSendMessage
# 載入 json 標準函式庫，處理回傳的資料格式
import requests, json, time, statistics

In [11]:
app = Flask(__name__)

access_token = 'Nkju9l/a0p1IXBDaeUpUABJhA3DY4ek/zS2pW3AMUYoagBuTXvHMEdrheBOuL3q5Ty/k6+11UFTihIzd5+SqciLoKNYI/gG0XM6cpW+jB0OlcbvqgSvWF1aD2GbWl96FW9tPNdeK0+TZobRLmeciOQdB04t89/1O/w1cDnyilFU='
channel_secret = 'fd20299c1f5e9dfc7a89330b458ce6b2'

@app.route("/", methods=['POST'])
def linebot():
  body = request.get_data(as_text=True)    # 取得收到的訊息內容
  try:
    line_bot_api = LineBotApi(access_token)               # 確認 token 是否正確
    handler = WebhookHandler(channel_secret)              # 確認 secret 是否正確
    signature = request.headers['X-Line-Signature']       # 加入回傳的 headers
    handler.handle(body, signature)                       # 綁定訊息回傳的相關資訊
    json_data = json.loads(body)                          # 轉換內容為 json 格式
    reply_token = json_data['events'][0]['replyToken']    # 取得回傳訊息的 Token ( reply message 使用 )
    user_id = json_data['events'][0]['source']['userId']  # 取得使用者 ID ( push message 使用 )
    print(json_data)                         # 印出內容
    if 'message' in json_data['events'][0]:           # 如果傳送的是 message
      if json_data['events'][0]['message']['type'] == 'location':                    # 如果 message 的類型是地圖 location
        address = json_data['events'][0]['message']['address'].replace('台','臺')   # 取出地址資訊，並將「台」換成「臺」
        reply_message(f'{address}\n\n{current_weather(address)}\n\n{forecast(address)}', reply_token, access_token)
        print(address)
      if json_data['events'][0]['message']['type'] == 'text':   # 如果 message 的類型是文字 text
        text = json_data['events'][0]['message']['text']      # 取出文字
        if text == '雷達回波圖' or text == '雷達回波':           # 如果是雷達回波圖相關的文字
                # 傳送雷達回波圖 ( 加上時間戳記 )
          reply_image(f'https://cwaopendata.s3.ap-northeast-1.amazonaws.com/Observation/O-A0058-001.png?{time.time_ns()}', reply_token, access_token)
        elif text == '地震資訊' or text == '地震':        # 如果是地震相關的文字
          msg = earth_quake()   # 爬取地震資訊
          push_message(msg[0], user_id, access_token)       # 傳送地震資訊 ( 用 push 方法，因為 reply 只能用一次 )
          reply_image(msg[1], reply_token, access_token)    # 傳送地震圖片 ( 用 reply 方法 )
        else:
          reply_message(text, reply_token, access_token)        # 如果是一般文字，直接回覆同樣的文字
  except Exception as e:
    print(f"Error: {e}")                       # 如果發生錯誤，印出 error
  return 'OK'                              # 驗證 Webhook 使用，不能省略

if __name__ == "__main__":
  run_with_ngrok(app)                        # 串連 ngrok 服務
  app.run()

# 地震資訊函式
def earth_quake():
  msg = ['找不到地震資訊','https://example.com/demo.jpg']            # 預設回傳的訊息
  try:
    code = 'CWA-C7DED748-AB01-46ED-B03D-3730BA31B6B0'
    url = f'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0016-001?Authorization={code}'
    e_data = requests.get(url)                                   # 爬取地震資訊網址
    e_data_json = e_data.json()                                  # json 格式化訊息內容
    eq = e_data_json['records']['Earthquake']                    # 取出地震資訊
    for i in eq:
      loc = i['EarthquakeInfo']['Epicenter']['Location']       # 地震地點
      val = i['EarthquakeInfo']['EarthquakeMagnitude']['MagnitudeValue'] # 地震規模
      dep = i['EarthquakeInfo']['FocalDepth']              # 地震深度
      eq_time = i['EarthquakeInfo']['OriginTime']              # 地震時間
      img = i['ReportImageURI']                                # 地震圖
      msg = [f'{loc}，芮氏規模 {val} 級，深度 {dep} 公里，發生時間 {eq_time}。', img]
      break     # 取出第一筆資料後就 break
    return msg    # 回傳 msg
  except:
    return msg    # 如果取資料有發生錯誤，直接回傳 msg

# LINE push 訊息函式
def push_message(msg, uid, token):
  headers = {'Authorization':f'Bearer {token}','Content-Type':'application/json'}
  body = {
  'to':uid,
  'messages':[{
      "type": "text",
      "text": msg
      }]
  }
  req = requests.request('POST', 'https://api.line.me/v2/bot/message/push', headers=headers,data=json.dumps(body).encode('utf-8'))
  print(req.text)

# LINE 回傳訊息函式
def reply_message(msg, rk, token):
  headers = {'Authorization':f'Bearer {token}','Content-Type':'application/json'}
  body = {
  'replyToken':rk,
  'messages':[{
      "type": "text",
      "text": msg
      }]
  }
  req = requests.request('POST', 'https://api.line.me/v2/bot/message/reply', headers=headers,data=json.dumps(body).encode('utf-8'))
  print(req.text)


# LINE 回傳圖片函式
def reply_image(msg, rk, token):
  headers = {'Authorization':f'Bearer {token}','Content-Type':'application/json'}
  body = {
  'replyToken':rk,
  'messages':[{
      'type': 'image',
      'originalContentUrl': msg,
      'previewImageUrl': msg
      }]
  }
  req = requests.request('POST', 'https://api.line.me/v2/bot/message/reply', headers=headers,data=json.dumps(body).encode('utf-8'))
  print(req.text)

# 目前天氣函式
def current_weather(address):
  city_list, area_list, area_list2 = {}, {}, {} # 定義好待會要用的變數
  msg = '找不到氣象資訊。'                         # 預設回傳訊息

  # 定義取得資料的函式
  def get_data(url):
    w_data = requests.get(url)   # 爬取目前天氣網址的資料
    w_data_json = w_data.json()  # json 格式化訊息內容
    location = w_data_json["records"]["Station"]  # 取出對應地點的內容
    for i in location:
      name = i["StationName"]                       # 測站地點
      city = i["GeoInfo"]["CountyName"]     # 縣市名稱
      area = i["GeoInfo"]["TownName"]     # 鄉鎮行政區
      temp = check_data(i["WeatherElement"]["AirTemperature"])                       # 氣溫
      humd = check_data(round(float(i["WeatherElement"]["RelativeHumidity"] ) ,1)) # 相對濕度
      if area not in area_list:
        area_list[area] = {'temp':temp, 'humd':humd}  # 以鄉鎮區域為 key，儲存需要的資訊
      if city not in city_list:
        city_list[city] = {'temp':[], 'humd':[]}       # 以主要縣市名稱為 key，準備紀錄裡面所有鄉鎮的數值
      city_list[city]['temp'].append(temp)   # 記錄主要縣市裡鄉鎮區域的溫度 ( 串列格式 )
      city_list[city]['humd'].append(humd)   # 記錄主要縣市裡鄉鎮區域的濕度 ( 串列格式 )

  # 定義如果數值小於 0，回傳 False 的函式
  def check_data(e):
    return False if float(e)<0 else float(e)

  # 定義產生回傳訊息的函式
  def msg_content(loc, msg):
    a = msg
    for i in loc:
      if i in address:  # 如果地址裡存在 key 的名稱
        temp = f"氣溫 {loc[i]['temp']} 度，" if loc[i]['temp'] != False else ''
        humd = f"相對濕度 {loc[i]['humd']}%，" if loc[i]['humd'] != False else ''
        description = f'{temp}{humd}'.strip('，')
        a = f'{description}。' # 取出 key 的內容作為回傳訊息使用
        break
    return a

  try:
        # 因為目前天氣有兩組網址，兩組都爬取
    code = 'CWA-C7DED748-AB01-46ED-B03D-3730BA31B6B0'
    get_data(f'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization={code}&format=JSON')
    get_data(f'https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0003-001?Authorization={code}&format=JSON')

    for i in city_list:
      if i not in area_list2: # 將主要縣市裡的數值平均後，以主要縣市名稱為 key，再度儲存一次，如果找不到鄉鎮區域，就使用平均數值
        area_list2[i] = {'temp':round(statistics.mean(city_list[i]['temp']),1),
                         'humd':round(statistics.mean(city_list[i]['humd']),1)
                          }
    msg = msg_content(area_list2, msg)  # 將訊息改為「大縣市」
    msg = msg_content(area_list, msg)   # 將訊息改為「鄉鎮區域」
    return msg    # 回傳 msg
  except:
    return msg    # 如果取資料有發生錯誤，直接回傳 msg

def forecast(address):
  area_list = {}
  # 將主要縣市個別的 JSON 代碼列出
  json_api = {"宜蘭縣":"F-D0047-001","桃園市":"F-D0047-005","新竹縣":"F-D0047-009","苗栗縣":"F-D0047-013",
              "彰化縣":"F-D0047-017","南投縣":"F-D0047-021","雲林縣":"F-D0047-025","嘉義縣":"F-D0047-029",
              "屏東縣":"F-D0047-033","臺東縣":"F-D0047-037","花蓮縣":"F-D0047-041","澎湖縣":"F-D0047-045",
              "基隆市":"F-D0047-049","新竹市":"F-D0047-053","嘉義市":"F-D0047-057","臺北市":"F-D0047-061",
              "高雄市":"F-D0047-065","新北市":"F-D0047-069","臺中市":"F-D0047-073","臺南市":"F-D0047-077",
              "連江縣":"F-D0047-081","金門縣":"F-D0047-085"}
  msg = '找不到天氣預報資訊。'    # 預設回傳訊息
  try:
    code = 'CWA-C7DED748-AB01-46ED-B03D-3730BA31B6B0'
    url = f'https://opendata.cwa.gov.tw/api/v1/rest/datastore/F-C0032-001?Authorization={code}&format=JSON'
    f_data = requests.get(url)   # 取得主要縣市預報資料
    f_data_json = f_data.json()  # json 格式化訊息內容
    location = f_data_json["records"]['location']  # 取得縣市的預報內容
    for i in location:
      city = i['locationName']    # 縣市名稱
      wx8 = i['weatherElement'][0]['time'][0]['parameter']['parameterName']    # 天氣現象
      mint8 = i['weatherElement'][2]['time'][0]['parameter']['parameterName']  # 最低溫
      maxt8 = i['weatherElement'][4]['time'][0]['parameter']['parameterName']  # 最高溫
      ci8 = i['weatherElement'][3]['time'][0]['parameter']['parameterName']    # 舒適度
      pop8 = i['weatherElement'][1]['time'][0]['parameter']['parameterName']   # 降雨機率
      area_list[city] = f'未來 8 小時{wx8}，最高溫 {maxt8} 度，最低溫 {mint8} 度，降雨機率 {pop8} %'  # 組合成回傳的訊息，存在以縣市名稱為 key 的字典檔裡
    for i in area_list:
      if i in address:        # 如果使用者的地址包含縣市名稱
        msg = area_list[i]  # 將 msg 換成對應的預報資訊
        # 將進一步的預報網址換成對應的預報網址
        url = f'https://opendata.cwa.gov.tw/api/v1/rest/datastore/{json_api[i]}?Authorization={code}&elementName=WeatherDescription'
        f_data = requests.get(url)  # 取得主要縣市裡各個區域鄉鎮的氣象預報
        f_data_json = f_data.json() # json 格式化訊息內容
        location = f_data_json['records']['locations'][0]['location']    # 取得預報內容
        break
    for i in location:
      city = i['locationName']   # 取得縣市名稱
      wd = i['weatherElement'][0]['time'][1]['elementValue'][0]['value']  # 綜合描述
      if city in address:           # 如果使用者的地址包含鄉鎮區域名稱
        msg = f'未來八小時天氣{wd}' # 將 msg 換成對應的預報資訊
        break
    return msg  # 回傳 msg
  except:
    return msg  # 如果取資料有發生錯誤，直接回傳 msg


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://ed71-34-106-222-238.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


<ipython-input-11-d9c757f9d7a3>:10: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)               # 確認 token 是否正確
<ipython-input-11-d9c757f9d7a3>:11: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(channel_secret)              # 確認 secret 是否正確
INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 15:04:34] "POST / HTTP/1.1" 200 -


Error: list index out of range
{'destination': 'Uc18de40248732fb00bd321e4d948f7f3', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '492182758198608343', 'quoteToken': 'AQ3jmeOnb6hHu6ceWUVgsKBh_vIosfWumyy9VjdgCUhvzc1RdnCXcbY2du8EQhYF3lMY-SsPo0mKraxcD1A6XXvlOX2yNHROKvaSbLkYEAbvACyW_CcUxAb_ZO1lJyFN7zazD4T5JShmqcHL4q9DCQ', 'text': '北投區'}, 'webhookEventId': '01HN0JDXE9DGSQ3GG8ZZGXAYED', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1706195088746, 'source': {'type': 'user', 'userId': 'U25d3c86ec67cd7c658eb48113ebd70d7'}, 'replyToken': '0909e12d5a45463f98742db809ee6371', 'mode': 'active'}]}


...

{"sentMessages":[{"id":"492182762191847601","quoteToken":"lYImOz2k02UZjqCeyOa47QNlxQTQT5Oko8hBg7TcHbyIfhB5D9P1Qpo5kpA_BJfqlXrk6ugoVgJWEhZB7viRDimAuZiZPAw6Gt7L2rTlEjr90Vn3kfBZbIKoIhE98qkJv4iu9peZP1S7j7m-7ZkNXA"}]}
{'destination': 'Uc18de40248732fb00bd321e4d948f7f3', 'events': [{'type': 'message', 'message': {'type': 'location', 'id': '492182781972185297', 'latitude': 25.112846, 'longitude': 121.511023, 'title': '千造環保興業', 'address': '台北市北投區自強街5巷116弄13號2樓'}, 'webhookEventId': '01HN0JEB4SGJZGSWDBNQCJ49YW', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1706195102853, 'source': {'type': 'user', 'userId': 'U25d3c86ec67cd7c658eb48113ebd70d7'}, 'replyToken': 'c2a8dd1e09b2499083303554057e3f43', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 15:05:06] "POST / HTTP/1.1" 200 -


{"sentMessages":[{"id":"492182787811180563","quoteToken":"Quk3oYE5sQIHIosOwju1IhTuyvDguglXANubskq7B7sss7gr_jd9vmcDY9dRD7DeEe2YLFvB0mP-Rnot2cQvdBBNcxGEfuLM-InOjfa0hdP-B9YKAwN3Yn3ku_pmusfqtQGpFQc3D3fFAkBKZrOdBA"}]}
臺北市北投區自強街5巷116弄13號2樓
{'destination': 'Uc18de40248732fb00bd321e4d948f7f3', 'events': [{'type': 'message', 'message': {'type': 'text', 'id': '492182882283159942', 'quoteToken': 'f56qa9KqPfuImSL0p4IFcnnL2C-hwZpZXf1gE5VKNV-BFkdy6Y7tGFC6XacKxGOslvrUN-b9Jqh20Ty-CAmOZNuseYzdpnaVe6oN37HAuXmJlslZ-iG7CFh7CFa7R3CooXgFpv1rE1SQx85MILMkCQ', 'text': '雷達回波'}, 'webhookEventId': '01HN0JG62F1DRRCDC1CDGYE87T', 'deliveryContext': {'isRedelivery': False}, 'timestamp': 1706195162737, 'source': {'type': 'user', 'userId': 'U25d3c86ec67cd7c658eb48113ebd70d7'}, 'replyToken': 'dcff197559434b159efe27fe8a706bcc', 'mode': 'active'}]}


INFO:werkzeug:127.0.0.1 - - [25/Jan/2024 15:06:04] "POST / HTTP/1.1" 200 -


{"sentMessages":[{"id":"492182884984815748","quoteToken":"lHIIjR6HnOHHofpihwc7vWuk6cyXaS11dKk-itaisp7BxWp5AJV6UBsoLBx-Gc_TbzgY0l7Be_y3xwHKJoxpLPA8j0BTywlyy5pR8IaTmPzIBTeqfOEt4g3umzzHvPqz6odoTYclxdFwjcBM0aPDGg"}]}
